In [ ]:
import sys
print(sys.version)

In [ ]:
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
%matplotlib inline

https://github.com/JanetMatsen/elvizAnalysis/blob/dc568af7ed589410bf9612ba5ec7ccf257e87d1e/elviz_pca.py

http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
sample_info = pd.read_csv('../data/sample_info.tsv', sep='\t')

In [ ]:
sample_info.head()

In [ ]:
all_X = pd.read_csv('../data/raw_data/3_summary_rpkm_byGeneProuct.xls', sep = '\t')
all_X.head(2)

In [ ]:
all_X.shape

In [ ]:
X = all_X.drop(['product'], axis=1).T
X.head()

In [ ]:
X.reset_index(inplace=True)

In [ ]:
X = X.rename(columns={'index':'sample'})

In [ ]:
X.head()

In [ ]:
X.head(3)

In [ ]:
X_counts = X.drop('sample', axis=1)

In [ ]:
X_counts.head(2)

In [ ]:
pca = PCA()
pca.fit(X_counts)

In [ ]:
variances = pd.DataFrame({'explained variance':pca.explained_variance_ratio_, 
                          'component':range(len(pca.explained_variance_ratio_))})

In [ ]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
variances.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
variances.plot.scatter(x='component', y='explained variance')

In [ ]:
transformed_X = pca.transform(X_counts)

In [ ]:
transformed_X.shape

In [ ]:
type(transformed_X)

In [ ]:
result = pd.DataFrame(transformed_X)

In [ ]:
result.head(2)

In [ ]:
X_2D = pd.DataFrame({'direction 1': transformed_X[:, 0],
                    'direction 2': transformed_X[:, 1]})

In [ ]:
print(X_2D.shape)
X_2D.head()

In [ ]:
X_2D = pd.concat([X_2D, pd.DataFrame({'sample': X['sample']})], axis=1)

In [ ]:
X_2D.head(2)

In [ ]:
sample_info.head(2)

In [ ]:
X_2D = X_2D.merge(sample_info, left_on = 'sample', right_on = 'ID')

In [ ]:
X_2D.head(3)

In [ ]:
X_2D.plot.scatter(x='direction 1', y='direction 2')

In [ ]:
def build_color_palette(num_items, weeks_before_switch):
    num_pre_switch_colors = weeks_before_switch
    num_post_switch_colors = num_items - num_pre_switch_colors
    print('preparing colors for {} pre-oxygen-switch'.format(
        num_pre_switch_colors),
          'samples and {} post-switch samples'
          .format(num_post_switch_colors))

    # get the first colors from this pallete:
    pre_switch_colors = \
        sns.cubehelix_palette(11, start=.5, rot=-.75)[0:num_pre_switch_colors]
    print(pre_switch_colors)

    # get post-switch colors here:
    # post_switch_colors = sns.diverging_palette(220, 20,
    # n=6)[::-1][0:num_post_switch_colors]
    post_switch_colors = \
        sns.color_palette("coolwarm", num_post_switch_colors)
    # sns.light_palette("navy", reverse=True)[0:num_post_switch_colors]
    rgb_colors = pre_switch_colors + post_switch_colors
    sns.palplot(rgb_colors)

    # check that we got the right amount
    print(num_items)
    assert (num_items == len(rgb_colors))
    print("")
    return rgb_colors

In [ ]:
X_2D.head(3)

In [ ]:
def plot_pca_results(plot_data, variances, facet_row=True, uniform_axes=True,
                     main_dir='./', plot_dir='./plots/',
                     savefig=False, figsize=None):
    
    # prepare axis labels, which also serve as dataframe column names.
    x_axis_label = 'principal component 1 ({0:.0%})'.format(variances[0])
    y_axis_label = 'principal component 2 ({0:.0%})'.format(variances[1])
    plot_data = plot_data.rename(columns={'direction 1':x_axis_label})
    plot_data = plot_data.rename(columns={'direction 2':y_axis_label})
    

    # define a custom color palette using:
    # Conditions were seized at week ten, so seven early samples in
    # the original condition and four latest samples in an alternative
    # condition.
    color_palette = build_color_palette(num_items=14 - 4 + 1,
                                        weeks_before_switch=7)
    # color_palette = sns.cubehelix_palette(11, start=.5, rot=-.75)

    # update matplotlib params for bigger fonts, ticks:
    mpl.rcParams.update({
        'font.size': 16, 'axes.titlesize': 17, 'axes.labelsize': 15,
        'xtick.labelsize': 10, 'ytick.labelsize': 13,
        'font.weight': 600,
        'axes.labelweight': 600, 'axes.titleweight': 600})
    # Plot with Seaborn
    if facet_row:
        if figsize is not None:
            print('warning: figsize not used if using facet_row')
        plt.figure(figsize=(4, 8))
    else:
        if figsize is not None:
            print('setting figure size to {}'.format(figsize))
            plt.figure(figsize=figsize)
        else:
            plt.figure(figsize=(6, 12))
    sns.set(style="ticks")

    # prepare the max and min axes values if we are forcing them to same range
    pc_colnames = [col for col in plot_data.columns
                   if 'principal component' in col]

    max_value = plot_data[pc_colnames].max(axis=0).max()
    min_value = plot_data[pc_colnames].min(axis=0).min()

    axis_max = math.ceil(max_value * 100) / 100.0
    axis_min = math.floor(min_value * 100) / 100.0

    def base_plot(**kwargs):
        plot = sns.FacetGrid(plot_data,
                             hue='week', palette=color_palette,
                             size=3, aspect=1,
                             **kwargs)
        plot = (plot.map(plt.scatter, x_axis_label, y_axis_label,
                         edgecolor="w", s=60).add_legend())
        return plot

    plot_args = {}

    if facet_row:
        plot_args['row'] = 'oxy'
        plot_args['col'] = 'rep'
    if uniform_axes:
        plot_args['xlim'] = (axis_min, axis_max)
        plot_args['ylim'] = (axis_min, axis_max)

    if len(plot_args) > 0:
        print(plot_args)
    
    g = base_plot(**plot_args)

    #filename = concat_dir_and_filename(
    #    plot_dir, 'pca_of_top_{}_percent--'.format(top_percent))
    filename = 'plot'

    # prepare a filename, depending on whether all taxonomy or only genus
    # is used.
    if uniform_axes:
        filename += '_unif_axes_'
    if facet_row:
        filename += '--faceted.pdf'
    else:
        filename += '.pdf'

    if savefig:
        g.fig.savefig(filename)

In [ ]:
plot_pca_results(plot_data = X_2D, variances = pca.explained_variance_ratio_, 
                 facet_row=True, uniform_axes=True)

In [ ]:
plot_pca_results(plot_data = X_2D, variances = pca.explained_variance_ratio_, 
                 facet_row=False, uniform_axes=True)

In [ ]:
plot_pca_results(plot_data = X_2D, variances = pca.explained_variance_ratio_, 
                 facet_row=False, uniform_axes=False, figsize=(20, 6))